In [1]:
import argparse
import collections

import numpy as np

import torch
import torch.optim as optim
from torchvision import transforms

from retinanet import model
from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
    Normalizer
from torch.utils.data import DataLoader

from retinanet import coco_eval
from retinanet import csv_eval

assert torch.__version__.split('.')[0] == '1'

print('CUDA available: {}'.format(torch.cuda.is_available()))

CUDA available: True


In [4]:
csv_inference = 'data/test_retinanet.csv'
csv_classes = 'data/class_retinanet.csv'
model_path = 'csv_retinanet_68.pt'
dataset_inference = CSVDataset(train_file=csv_inference, class_list=csv_classes,
                                   transform=transforms.Compose([Normalizer(), Resizer()]))

sampler = AspectRatioBasedSampler(dataset_inference, batch_size=1, drop_last=False)
dataloader_inference = DataLoader(dataset_inference, num_workers=3, collate_fn=collater, batch_sampler=sampler)
use_gpu = True

retinanet = torch.load(model_path)
retinanet = retinanet.cuda()
retinanet.training = False
retinanet.eval()

detect, style = csv_eval._get_detections(dataset_inference, retinanet)

In [11]:
print(len(detect))
print(len(detect[0]))
print(len(detect[0][1]))

219
14
2


In [12]:
detect[0][1]

array([[3.97607025e+02, 5.22449219e+02, 5.74884155e+02, 6.49325073e+02,
        1.78484112e-01],
       [4.13405701e+02, 5.81959167e+02, 5.51283020e+02, 6.59198608e+02,
        9.94595811e-02]])

In [13]:
detect[0][2]

array([[3.83589905e+02, 4.93742096e+02, 5.77356262e+02, 6.58654236e+02,
        1.09485075e-01],
       [3.90395142e+02, 5.31563660e+02, 5.69669739e+02, 7.76171265e+02,
        7.19911382e-02]])

In [18]:
SUB_ELEMENTS = {
    'arco-herradura':1,
    'dintel-adovelado':2,
    'arco-lobulado':3,
    'arco-medio-punto':4,
    'arco-apuntado':5,
    'vano-adintelado':6,
    'fronton':7,
    'arco-conopial':8,
    'arco-trilobulado':9,
    'serliana':10,
    'ojo-de-buey':11,
    'fronton-curvo':12,
    'fronton-partido':13,
    'columna-salomonica':14
}

In [31]:
detection_folder = 'detections'
gt_folder = 'gt'
import os, csv
os.makedirs(detection_folder, exist_ok=True)
os.makedirs(gt_folder, exist_ok=True)

labels=list(SUB_ELEMENTS.keys())

for j in range(len(detect)):
    detection_image = detect[j]
    image_name = dataset_inference.image_names[j].split('/')[-1][:-4]
    annot = dataset_inference.load_annotations(j)[0]
    saved_gt = []
    for g in range(len(annot)):
        local_annot = annot[g]
        local_gt = []
        local_gt.append(labels[int(local_annot[-1])])
        local_gt.append(int(local_annot[0]))
        local_gt.append(int(local_annot[1]))
        local_gt.append(int(local_annot[2]))
        local_gt.append(int(local_annot[3]))
        saved_gt.append(local_gt)
    saved_detection = []
    for k in range(len(detection_image)):
        local_detect = detection_image[k]
        label_name = labels[k]
        for l in range(len(local_detect)):
            local_saved_detect = []
            relevant_detect = local_detect[l]
            local_saved_detect.append(label_name)
            local_saved_detect.append(relevant_detect[-1])
            local_saved_detect.append(relevant_detect[0])
            local_saved_detect.append(relevant_detect[1])
            local_saved_detect.append(relevant_detect[2])
            local_saved_detect.append(relevant_detect[3])
            saved_detection.append(local_saved_detect)
    with open(os.path.join(detection_folder,image_name + '.txt'), 'w') as f:
            writer = csv.writer(f, delimiter=' ')
            writer.writerows(saved_detection)
    with open(os.path.join(gt_folder,image_name + '.txt'), 'w') as f:
            writer = csv.writer(f, delimiter=' ')
            writer.writerows(saved_gt)
        